In [ ]:
!pip install langchain

In [ ]:
!pip install openai

In [3]:
!pip install langchain-community

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(
    openai_api_base = "https://openrouter.ai/api/v1",
    openai_api_key = "<OPEN_ROUTER_API_KEY>",
    model = "MODEL_NAME",
    temperature=0.9) #max_tokens=100openrouter - only for colab user

Removing Think area

In [ ]:
import re

def remove_think_sections(text):
    return re.sub(r"<think>.*?</think>", "", text)

Basic Run

In [ ]:
text="what are the 5 industries in thanjavur?"

In [4]:
output = llm.predict(text)
output = remove_think_sections(output)
print(output)

Prompt Template

In [6]:
from langchain.prompts import PromptTemplate

template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."

prompt = PromptTemplate(
    input_variables=['cuisine'],
    template=template
)

formatted_prompt = prompt.format(cuisine="Chinese-sichuang-Tang")
print(formatted_prompt)

Few Shot Prompt

In [7]:
from langchain.prompts import FewShotPromptTemplate


examples = [
    {"word": "cat", "definition": "A small domesticated carnivorous mammal."},
    {"word": "dog", "definition": "A domesticated carnivorous mammal that typically has a long snout."},
]

example_prompt = PromptTemplate(
    input_variables=["word", "definition"],
    template="Word: {word}\nDefinition: {definition}\n"
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Keep the answer very short and consise and Define the following words:",
    suffix="Word: {input}\nDefinition:",
    input_variables=["input"],
)

output = (few_shot_prompt.format(input="Lion"))
print (output)

In [5]:
output = llm.predict(output)
output = remove_think_sections(output)
print(output)

Simple Chain

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI


prompt = PromptTemplate(
    input_variables=["topic"],
    template="Give me a tweet idea about {topic}"
)

chain = LLMChain(llm=llm, prompt=prompt)

response = chain.run("AI in education")
response = remove_think_sections(response)
print(response)

Sequential Chain

In [ ]:
from langchain.chains import SequentialChain

# Prompt 1: Generate title
title_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Write a compelling blog post title about {topic}. only give one one title as the output nothing more than that"
)
title_chain = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

# Prompt 2: Use title to generate intro
intro_prompt = PromptTemplate(
    input_variables=["title"],
    template="Write a short blog introduction for the post titled: {title}. only give the introduction as the output nothing more than that. have bullet points and emojis in the intro"
)
intro_chain = LLMChain(llm=llm, prompt=intro_prompt, output_key="intro")

# Chain them together
overall_chain = SequentialChain(
    chains=[title_chain, intro_chain],
    input_variables=["topic"],
    output_variables=["title", "intro"]
)

result = overall_chain.invoke("LangChain for AI-powered apps")
print(result)

In [ ]:
print(result["intro"])

Custom Template

In [ ]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import BaseOutputParser


memory = ConversationBufferMemory()

class RemoveThinkParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        # remove <think>...</think> blocks
        return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()

custom_prompt = PromptTemplate(
    input_variables=["history", "input"],
    template="""
You are a helpful and concise assistant.

{history}
Human: {input}
AI:"""
)

conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True,
    prompt=custom_prompt,
    output_parser=RemoveThinkParser()
)


print (conversation.predict(input="Hi, my name is Deepan."))


print (conversation.predict(input="What’s my name?"))

